# Problem 14.05 Required Power for R134a Compression Using a High Precision Equation of State

Refrigerant R134a is compressed from a saturated vapor at 5 °C to an outlet pressure of 1 MPa. Calculate the power of the compressor, using a high-precision EOS.

The mechanical efficiency is 0.95, and the isentropic efficiency 0.7; the mass flow rate is 3000 kg/hr.

## Solution

This is straightforward.

In [1]:
# Set the conditions and imports
from scipy.constants import bar, hour
from thermo import ChemicalConstantsPackage, PRMIX, CEOSLiquid, CoolPropLiquid, CEOSGas, CoolPropGas, FlashPureVLS
fluid = 'R134a'
constants, correlations = ChemicalConstantsPackage.from_IDs([fluid])

T1 = 5 + 273.15
VF1 = 1
P2 = 10*bar
zs = [1]
eta_isentropic = 0.7
eta_mechanical = 0.9

In [2]:
backend = 'HEOS'
gas = CoolPropGas(backend, fluid, T=T1, P=1e5, zs=zs)
liquid = CoolPropLiquid(backend, fluid, T=T1, P=1e5, zs=zs)

flasher = FlashPureVLS(constants, correlations, gas=gas, liquids=[liquid], solids=[])

# Flash at inlet conditions to obtain initial enthalpy
state_1 = flasher.flash(T=T1, VF=VF1)
# Flash at outlet condition - entropy is conserved by compressors and expanders!
state_2_ideal = flasher.flash(S=state_1.S(), P=P2)
# Compute the change in enthalpy
delta_H_ideal = (state_2_ideal.H()-state_1.H())
# The definition of isentropic efficiency means that the actual amount of heat added is
# dH_actual = dH_idea/eta_isentropic
H_added_to_fluid_actual = delta_H_ideal/eta_isentropic

state_2 = flasher.flash(H=state_1.H() + H_added_to_fluid_actual, P=P2)

# To compute the actual power, itis more convinient to use the mass enthalpy
actual_power_per_kg = (state_2.H_mass() - state_1.H_mass())/(eta_mechanical) # W/kg
actual_power = actual_power_per_kg*3000/hour
print(f'The actual power is {actual_power:.0f} W')
print(f'The actual outlet temperature is {state_2.T: .2f} K')

The actual power is 28858 W
The actual outlet temperature is  324.80 K
